# Summerize Shareholding:

#### Summerize: 

- For all stocks from RawReports folder, read specific Excels sheet into dataframe  
- Add stock name column to df 
- Transform the df 
- Consolidate the df 

#### Analyze:

- Compute something
- Write the computed new data 
- Analyze or visualize the new data 

#### Summary Report File Name Convention: 
- Summary01-Top-Ratios 
- Summary02-Quarterly-Results
- Summary03-Profit-And-Loss
- Summary04-Compounded-Sales-Growth
- Summary05-Compounded-Profit-Growth
- Summary06-Stock-Price-CAGR 
- Summary07-Return-on-Equity
- Summary08-Balance-Sheet
- Summary09-Cash-Flows
- Summary10-Ratios
- Summary11-Shareholding-Pattern

## 1.1 Imports:

In [1]:
import requests as rq
import pandas as pd 
import time
import datetime
from bs4 import BeautifulSoup
import re
import os


## 1.2 Define Inputs:

>  ⚠️ <span style="color:red"> **Update the CSV File below to read as needed:**</span>

In [2]:
## CAREFULLY UPDATE THIS. This should be same as what specified in RawReport notebook

# For My Watchlisted stocks
input_cvs_file = "../Input/" + "stocks-watchlisted.csv"
# For PSU stocks
input_cvs_file = "../Input/" + "stocks-psu.csv"

input_cvs_file = "../Input/" + "stocks-watchlisted.csv"

## UPDATE THIS. The folder is in  ../Output/RawReports/. This is where raw reports are read and summerized

raw_report_folder_name     = "20240817-132127"


In [3]:
raw_report_folder_path = "../Output/RawReports/" + raw_report_folder_name

df = pd.read_csv(input_cvs_file)
df.head(5)


,Screener Stock Symbol,Url Segment
0,526433,consolidated
1,532407,consolidated
2,544021,consolidated
3,AARTIDRUGS,consolidated
4,AARTIIND,consolidated


## 1.3 Process reports:

In [4]:
df_quaterly_results = pd.DataFrame([])

for index, row in df.iterrows():
    stock_symbol = row["Screener Stock Symbol"]
    filepath = raw_report_folder_path + "/" + stock_symbol + ".xlsx"
    df_data =  pd.read_excel(filepath, sheet_name="Quarterly Results")
    print("Processing stock ",stock_symbol)
    # Add stockname column and move into 2nd position: 
    df_data['Stockname'] = stock_symbol
    column_to_move = df_data.pop("Stockname")
    df_data.insert(1, "Stockname", column_to_move)
    df_quaterly_results = pd.concat([df_quaterly_results, df_data], ignore_index=True, sort=False)
    
print("All done!")    

Processing stock  526433
Processing stock  532407
Processing stock  544021
Processing stock  AARTIDRUGS
Processing stock  AARTIIND
Processing stock  AARTIPHARM
Processing stock  AARTISURF
Processing stock  ABB
Processing stock  ACC
Processing stock  ADANIENT
Processing stock  AFFLE
Processing stock  ALKYLAMINE
Processing stock  ANGELONE
Processing stock  ARE&M
Processing stock  ASAHIINDIA
Processing stock  ASHOKLEY
Processing stock  ASIANPAINT
Processing stock  ASTRAL
Processing stock  AXISBANK
Processing stock  BAJAJ-AUTO
Processing stock  BAJAJELEC
Processing stock  BAJAJFINSV
Processing stock  BAJFINANCE
Processing stock  BALAMINES
Processing stock  BATAINDIA
Processing stock  BEL
Processing stock  BERGEPAINT
Processing stock  BHARTIARTL
Processing stock  BOROLTD
Processing stock  BRITANNIA
Processing stock  BSE
Processing stock  CAMS
Processing stock  CAMPUS
Processing stock  CDSL
Processing stock  CENTUM
Processing stock  CGPOWER
Processing stock  COCHINSHIP
Processing stock  COFO

## 1.5 Inspect dataframe:

In [5]:
print(len(df_quaterly_results))

1940


In [6]:
df_quaterly_results.describe()

,Quarterly Results,Stockname,Jun 2021,Sep 2021,Dec 2021,Mar 2022,Jun 2022,Sep 2022,Dec 2022,Mar 2023,Jun 2023,Sep 2023,Dec 2023,Mar 2024,Jun 2024,Mar 2021
count,1940,1940,1601,1601,1623,1644,1697,1698,1734,1746,1756,1746,1755,1755,1746,33
unique,17,160,938,930,956,983,974,1005,1032,1056,1040,1067,1054,1086,1063,31
top,Tax %,KOTAKBANK,0,26%,26%,26%,0,0,26%,26%,25%,25%,26%,0,26%,10
freq,160,14,31,29,34,25,33,30,28,32,38,39,36,25,38,2


In [7]:
df_quaterly_results.columns

Index(['Quarterly Results', 'Stockname', 'Jun 2021', 'Sep 2021', 'Dec 2021',
       'Mar 2022', 'Jun 2022', 'Sep 2022', 'Dec 2022', 'Mar 2023', 'Jun 2023',
       'Sep 2023', 'Dec 2023', 'Mar 2024', 'Jun 2024', 'Mar 2021'],
      dtype='object')

## 1.6 Sort column ordering:

In [8]:
date = '1970-01-01'
df_quaterly_results = df_quaterly_results.sort_index(
    axis=1, 
    key=lambda x: pd.to_datetime(x, errors='coerce', format='mixed').fillna(date)
)

In [9]:
df_quaterly_results.columns

Index(['Quarterly Results', 'Stockname', 'Mar 2021', 'Jun 2021', 'Sep 2021',
       'Dec 2021', 'Mar 2022', 'Jun 2022', 'Sep 2022', 'Dec 2022', 'Mar 2023',
       'Jun 2023', 'Sep 2023', 'Dec 2023', 'Mar 2024', 'Jun 2024'],
      dtype='object')

In [10]:
df_quaterly_results.head(5)

,Quarterly Results,Stockname,Mar 2021,Jun 2021,Sep 2021,Dec 2021,Mar 2022,Jun 2022,Sep 2022,Dec 2022,Mar 2023,Jun 2023,Sep 2023,Dec 2023,Mar 2024,Jun 2024
0,Sales,526433,NaN,41.64,47.70,47.62,54.71,50.92,63.42,54.14,51.94,50.95,50.39,55.72,44.59,52.62
1,Expenses,526433,NaN,36.70,39.98,42.30,51.69,45.54,57.43,48.94,49.74,48.58,48.46,55.37,45.53,48.43
2,Operating Profit,526433,NaN,4.94,7.72,5.32,3.02,5.38,5.99,5.20,2.20,2.37,1.93,0.35,-0.94,4.19
3,OPM %,526433,NaN,11.86%,16.18%,11.17%,5.52%,10.57%,9.44%,9.60%,4.24%,4.65%,3.83%,0.63%,-2.11%,7.96%
4,Other Income,526433,NaN,0.50,0.35,3.89,2.19,2.43,1.58,2.74,1.55,1.98,0.42,1.14,3.31,3.20


## 1.7 Save summary report:

In [11]:
output_path = "../Output/SummaryReports/"+"Summary02-Quarterly-Results-"+datetime.datetime.now().strftime("%Y%m%d-%H%M%S")+".csv"

df_quaterly_results.to_csv(output_path,index=False,sep=",")